## Syncing your Fork

GitHub documentation: [Syncing a fork](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/syncing-a-fork)

---

# Pandas Demonstration

In this demonstration we will look at (the first part) of a common pattern in bioinformatics:
filenames from experimental data (such as FASTA, or in this case LCMS) contain the 
experimental design information.

We shall look into workflows to process the larger data components in module 3 of this course.

In [1]:
import numpy as np
import pandas as pd

# from numpy.random import default_rng
# rng = default_rng()

import matplotlib.pyplot as plt
%matplotlib inline

Here I copy a tidy set of data from the pandas documentation.

In [2]:
import datetime

df = pd.DataFrame({
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)] + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
})

df

,A,B,C,D,E,F
0,one,A,foo,1.636834,0.302295,2013-01-01
1,one,B,foo,0.127274,0.350630,2013-02-01
2,two,C,foo,-1.501619,-0.967004,2013-03-01
3,three,A,bar,-0.470206,0.667923,2013-04-01
4,one,B,bar,0.252735,-0.741901,2013-05-01
5,one,C,bar,-0.013764,1.361698,2013-06-01
6,two,A,foo,-0.975920,0.895060,2013-07-01
7,three,B,foo,-1.783712,1.896859,2013-08-01
8,one,C,foo,0.394234,2.053124,2013-09-01
9,one,A,bar,0.128505,-0.694331,2013-10-01


It is not uncommon to be provided an excel sheet that ends up looking something like the following
result of the call to `pd.pivot()`.

In [3]:
non_tidy_df = pd.pivot_table(df, index=["A", "B"], columns=["C"])
non_tidy_df

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -0.046739  1.320071  0.075722 -0.449437
      B -0.195837  0.615029 -0.654030  0.027494
      C -0.463411 -0.251023  0.130299  0.453537
three A -0.501121       NaN  0.452043       NaN
      B       NaN -0.966897       NaN  0.290959
      C -1.286020       NaN -0.750488       NaN
two   A       NaN  0.140881       NaN  0.390336
      B  0.031587       NaN -1.435929       NaN
      C       NaN -0.116339       NaN -0.716756

The solution to this is to stack the data:

In [4]:
non_tidy_df.stack()

D         E
A     B C                      
one   A bar -0.046739  0.075722
        foo  1.320071 -0.449437
      B bar -0.195837 -0.654030
        foo  0.615029  0.027494
      C bar -0.463411  0.130299
        foo -0.251023  0.453537
three A bar -0.501121  0.452043
      B foo -0.966897  0.290959
      C bar -1.286020 -0.750488
two   A foo  0.140881  0.390336
      B bar  0.031587 -1.435929
      C foo -0.116339 -0.716756

## Wrangling Data

Read our data file!

In [4]:
%%bash
ls data

filenames.txt


In [5]:
%%bash
# less data/filenames.txt

In pure Python we can read a file line by line, like this:

In [12]:
with open('data/filenames.txt', 'r') as file:
    print(file.readline())

02042021/Blank-r001.d



Convenience functions are provided in pandas for many common data types. Which type you
interact with most often will depend on the source of your data. 

I would hazard to guess that you will see text or .csv (comma separated value) files most often,
and that is what we will examine here.

I have copied the filenames from a current project and provided them, we shall try to prepare
the sample annotation data from these names and the provided specification.

### Provided Specification

```
[Grape Varitety]_[Smoke Event]_[Replicate]_[Treatment]
```

This means our output should be a data frame with (at least) those four columns.

In [65]:
data = pd.read_csv('data/filenames.txt', header = None, names = ['data']) ##can read not csv files, but bad things can happen
#data1 = pd.read_csv("data/filenames.txt", sep= '/', header = None) 

#expand around dates
data = data['data'].str.split('/', expand = True)
data.columns = ['date', 'filename']

#expand around specifications
data['filename'] = data['filename'].str.rstrip('.d')
temp_df = data['filename'].str.split("_", expand = True)

#merge
data = data.merge(temp_df, left_index = True, right_index = True)

#data['filename'].str.slice(0,2) == "ME"

data.columns = ['date', 'filename', 'grape', 'smoke', 'rep', 'treat', 'unknown']

#data['grape'].unique()

#data.dropna(subset = ['smoke','rep', 'treat'], how = 'all')

to_drop = data[['smoke', 'rep', 'treat']].isna().all(axis = 1)
data.loc[~to_drop]  ##~ an - flip a bool

,date,filename,grape,smoke,rep,treat,unknown
5,02042021,ME_S2C_R1_0,ME,S2C,R1,0,None
6,02042021,ME_S2C_R1_120,ME,S2C,R1,120,None
7,02042021,ME_S2C_R1_180,ME,S2C,R1,180,None
8,02042021,ME_S2C_R1_240,ME,S2C,R1,240,None
9,02042021,ME_S2C_R1_30,ME,S2C,R1,30,None
...,...,...,...,...,...,...,...
571,03232021,ME_S3C_R3_240,ME,S3C,R3,240,None
572,03232021,ME_S3C_R3_30,ME,S3C,R3,30,None
573,03232021,ME_S3C_R3_60,ME,S3C,R3,60,None
574,03232021,ME_S3C_R3_90,ME,S3C,R3,90,None


,date,filename,grape,smoke,rep,treat,unknown
5,02042021,ME_S2C_R1_0,ME,S2C,R1,0,None
6,02042021,ME_S2C_R1_120,ME,S2C,R1,120,None
7,02042021,ME_S2C_R1_180,ME,S2C,R1,180,None
8,02042021,ME_S2C_R1_240,ME,S2C,R1,240,None
9,02042021,ME_S2C_R1_30,ME,S2C,R1,30,None
...,...,...,...,...,...,...,...
571,03232021,ME_S3C_R3_240,ME,S3C,R3,240,None
572,03232021,ME_S3C_R3_30,ME,S3C,R3,30,None
573,03232021,ME_S3C_R3_60,ME,S3C,R3,60,None
574,03232021,ME_S3C_R3_90,ME,S3C,R3,90,None


,0,1
0,2042021,Blank-r001.d
1,2042021,Blank-r002.d
2,2042021,cONTROL 2.d
3,2042021,Control-r001.d
4,2042021,Control-r002.d
...,...,...
573,3232021,ME_S3C_R3_60.d
574,3232021,ME_S3C_R3_90.d
575,3232021,ME_S3C_R3_CO.d
576,3232021,Water blank-r001.d
